## Preprocessing

In [161]:
import sklearn
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
import numpy as np
from  matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline

from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np

In [162]:
%matplotlib inline
import seaborn as sns

In [163]:
train = pd.read_csv('./titanic/train.csv')
test = pd.read_csv('./titanic/test.csv')

In [164]:
myimputer_mean=Imputer(missing_values='NaN', strategy='mean', axis=0, verbose=0, copy=True)
myimputer_mf=Imputer(missing_values='NaN', strategy='most_frequent', axis=0, verbose=0, copy=True)

In [165]:
if 'Survived' in train.columns:  # Del result column, qual for the second run
    y=train['Survived']
    del train['Survived']  

In [166]:
print(y.head())

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64


In [167]:
le_pclass = preprocessing.LabelEncoder()
le_sex = preprocessing.LabelEncoder()
le_embarked = preprocessing.LabelEncoder()


In [168]:
le_flc = preprocessing.LabelEncoder()

интересно было попробывать замену категориальных фич с помощью способа описанного для catboost

In [169]:
class СatReplaceByP:      
    """  X.columnname = [cat,cat,dog]
         y = [1,0,0]
         catrepl = СatReplaceByP() 
         catrepl.fit(X.columnname, y)
         X.columnname = catrepl.transform(X.columnname)
         print(X.columnname)
         $[0.5,0.5,0]
         
         
         Replacement category for example "cat" by P(y=1)|P(x=cat)"""
    def __init__(self):
        self.replacment={}
    def fit(self, df_input, y_input):
        if df_input.shape != y_input.shape:
            print("Shape of inputs must be same")
            return 1
        df = pd.Series(df_input)
        y = pd.Series(y_input)
        catslist=df.groupby(df.values).min()
        grp = pd.concat([df, y], axis=1)
        grp.loc[grp[ grp.columns[0]]==0]
        for cat in catslist:
            #df.loc[df==cat] = len(grp.loc[(grp[grp.columns[0]]==cat) &(grp[grp.columns[1]]==1)])/len(grp.loc[grp[grp.columns[0]]==cat])
            self.replacment[cat] = len(grp.loc[(grp[grp.columns[0]]==cat) &(grp[grp.columns[1]]==1)])/len(grp.loc[grp[grp.columns[0]]==cat])
        #return df
    def params(self):
        return(self.replacment)
    def tramsform(self, df_input):
        df = pd.Series(df_input)
        catslist=df.groupby(df.values).min()
        for cat in catslist:
            #df.loc[df==cat] = len(grp.loc[(grp[grp.columns[0]]==cat) &(grp[grp.columns[1]]==1)])/len(grp.loc[grp[grp.columns[0]]==cat])
            if cat in self.replacment:
                df.loc[df==cat] = self.replacment[cat]
            else:
                print("Warning: The categorial {} is new in transform, replaced by 0".format(cat))
                df.loc[df==cat] = 0
        return df
    

In [170]:
train['is_test'] = False
test['is_test'] = True

In [171]:
X = pd.concat([train, test], axis=0)
X.index = range(len(X))
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
is_test        1309 non-null bool
dtypes: bool(1), float64(2), int64(4), object(5)
memory usage: 113.8+ KB


In [172]:
X['Embarked'] = X['Embarked'].astype(str)

In [173]:
X['Pclass'] = le_pclass.fit_transform(X['Pclass'])
X['Sex'] = le_sex.fit_transform(X['Sex'])
X['Embarked'] = le_embarked.fit_transform(X['Embarked'])

In [174]:
X['Sex'] =X['Sex'].astype(int)

In [175]:
#fill NaN mean value
X[['Age', 'SibSp','Parch',  'Fare']]=myimputer_mean.fit_transform(X[['Age', 'SibSp','Parch',  'Fare']])

In [176]:
#fill NaN most frequntly value
X[['Pclass', 'Sex','Embarked']] = myimputer_mf.fit_transform(X[['Pclass', 'Sex', 'Embarked']])

In [177]:
# after imputer values in float,convert 1.0 to 1
X.Sex = X.Sex.astype(int)
X.Embarked= X.Embarked.astype(int)
X.Pclass = X.Pclass.astype(int)

In [178]:
X.Sex.head() # check

0    1
1    0
2    0
3    0
4    1
Name: Sex, dtype: int32

In [179]:
#Replacement in catboost style

In [180]:

catrepl_emb = СatReplaceByP()
catrepl_sex = СatReplaceByP()
catrepl_class = СatReplaceByP()

In [ ]:
catrepl_emb.fit(X.Embarked.loc[~X.is_test], y)
catrepl_sex.fit(X.Sex.loc[~X.is_test], y)
catrepl_class.fit(X.Pclass.loc[~X.is_test], y)

перед тем как заменять категории создадим копию для собственно catboost пусть сделает это сам, тем более что пробовал очень разные варианты

In [181]:
X_cat=X.copy() 

In [182]:
X_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int32
Name           1309 non-null object
Sex            1309 non-null int32
Age            1309 non-null float64
SibSp          1309 non-null float64
Parch          1309 non-null float64
Ticket         1309 non-null object
Fare           1309 non-null float64
Cabin          295 non-null object
Embarked       1309 non-null int32
is_test        1309 non-null bool
dtypes: bool(1), float64(4), int32(3), int64(1), object(3)
memory usage: 98.5+ KB


In [183]:
X.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_test
0,1,2,"Braund, Mr. Owen Harris",1,22.0,1.0,0.0,A/5 21171,7.2500,NaN,2,False
1,2,0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1.0,0.0,PC 17599,71.2833,C85,0,False


In [184]:
X['Pclass2'] = catrepl_emb.tramsform(X.Pclass)
X['Sex2'] = catrepl_sex.tramsform(X.Sex)
X['Embarked2'] = catrepl_emb.tramsform(X.Embarked)

In [185]:
X.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_test,Pclass2,Sex2,Embarked2
0,1,0,"Braund, Mr. Owen Harris",0,22.0,1.0,0.0,A/5 21171,7.2500,NaN,0,False,0,0,0
1,2,0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1.0,0.0,PC 17599,71.2833,C85,0,False,0,0,0


In [186]:
X= pd.get_dummies(X, columns=['Sex', 'Pclass', 'Embarked'])


In [187]:
X.head(2)

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,is_test,Pclass2,Sex2,Embarked2,Sex_0,Pclass_0,Embarked_0
0,1,"Braund, Mr. Owen Harris",22.0,1.0,0.0,A/5 21171,7.2500,NaN,False,0,0,0,1,1,1
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1.0,0.0,PC 17599,71.2833,C85,False,0,0,0,1,1,1


способ, когда мы разносим категории в разные столбцы, но вместо единиц в столбцах ставим вероятность класса=1 в этой категории - дает лучший результат

In [518]:
X.Pclass_0 = X.Pclass_0 * X.Pclass2
X.Pclass_1 = X.Pclass_1 * X.Pclass2
X.Pclass_2 = X.Pclass_2 * X.Pclass2

In [219]:
X.Sex_0 = X.Sex_0 * X.Sex2
X.Sex_1 = X.Sex_1 * X.Sex2

In [220]:
X.Embarked_0 = X.Embarked_0 * X.Embarked2
X.Embarked_1 = X.Embarked_1 * X.Embarked2
X.Embarked_2 = X.Embarked_2 * X.Embarked2
X.Embarked_3 = X.Embarked_3 * X.Embarked2

In [221]:
X.head(2)


,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,is_test,Pclass2,...,Embarked2,Sex_0,Sex_1,Pclass_0,Pclass_1,Pclass_2,Embarked_0,Embarked_1,Embarked_2,Embarked_3
0,1,"Braund, Mr. Owen Harris",22.0,1.0,0.0,A/5 21171,7.2500,NaN,False,0.336957,...,0.336957,0.000000,0.188908,0.000000,0.0,0.336957,0.000000,0.0,0.336957,0.0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1.0,0.0,PC 17599,71.2833,C85,False,0.553571,...,0.553571,0.742038,0.000000,0.553571,0.0,0.000000,0.553571,0.0,0.000000,0.0


In [222]:
X_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int32
Name           1309 non-null object
Sex            1309 non-null int32
Age            1309 non-null float64
SibSp          1309 non-null float64
Parch          1309 non-null float64
Ticket         1309 non-null object
Fare           1309 non-null float64
Cabin          295 non-null object
Embarked       1309 non-null int32
is_test        1309 non-null bool
dtypes: bool(1), float64(4), int32(3), int64(1), object(3)
memory usage: 98.5+ KB


In [223]:
X.drop(['PassengerId', 'Name', 'Cabin', 'Ticket', 'Sex2', 'Embarked2','Pclass2' ], axis=1, inplace=True)


In [224]:
X_cat.drop(['PassengerId', 'Name', 'Cabin', 'Ticket' ], axis=1, inplace=True)

In [225]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
Age           1309 non-null float64
SibSp         1309 non-null float64
Parch         1309 non-null float64
Fare          1309 non-null float64
is_test       1309 non-null bool
Sex_0         1309 non-null float64
Sex_1         1309 non-null float64
Pclass_0      1309 non-null float64
Pclass_1      1309 non-null float64
Pclass_2      1309 non-null float64
Embarked_0    1309 non-null float64
Embarked_1    1309 non-null float64
Embarked_2    1309 non-null float64
Embarked_3    1309 non-null float64
dtypes: bool(1), float64(13)
memory usage: 134.3 KB


In [226]:
X_train = X.loc[X.is_test==False]
X_test = X.loc[X.is_test==True]

In [227]:
del X_test['is_test']
del X_train['is_test']

## Model Learning

In [228]:
import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import SGDClassifier

def cross_val_predict_proba(estimator, X_train, y_train):
    kfold = KFold(n_splits=5, shuffle=True)
    return cross_val_predict(estimator, X_train, y_train, cv=kfold, method='predict_proba')


In [531]:
#Random Forest the best
pipline_rf=Pipeline([('scale', StandardScaler()), 
                  ('model', RandomForestClassifier(n_estimators=10, 
                                                   #loss='deviance', 
                                                   #learning_rate =0.1,
                                                   random_state=1111))])
kfold_out=KFold(n_splits=4, shuffle=True)


depths = np.arange(5, 15, 2)
min_samples_leaf = np.arange(1, 10, 2)
max_leaf_nodes = [20, 50, 100, None]
min_weight_fraction_leaf = [0.0 ,0.1, 0.3, 0.5 ]
class_weight = ['balanced', None]
grid = {'model__max_depth': [3,4,6], 
        'model__max_leaf_nodes': max_leaf_nodes,
       'model__min_samples_leaf': [1,2,5], 
       #'model__min_samples_split': [2,3,4] , 
       # 'model__min_weight_fraction_leaf': min_weight_fraction_leaf,
       #'model__class_weight': class_weight,
         #'model__max_features' : [ 0.5,0.8, 'auto'],
        'model__n_estimators' : [150,170]}  
gridsearch = GridSearchCV(pipline_rf, grid, scoring='accuracy', cv=kfold_out)

In [532]:
gridsearch.fit(X_train,y)

GridSearchCV(cv=KFold(n_splits=4, random_state=None, shuffle=True),
       error_score='raise',
       estimator=Pipeline(steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('model', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=1111,
            verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'model__max_depth': [3, 4, 6], 'model__max_leaf_nodes': [20, 50, 100, None], 'model__min_samples_leaf': [1, 2, 5], 'model__n_estimators': [150, 170]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [533]:
gridsearch.best_params_

{'model__max_depth': 3,
 'model__max_leaf_nodes': 20,
 'model__min_samples_leaf': 1,
 'model__n_estimators': 170}

In [534]:
gridsearch.cv_results_.keys()

dict_keys(['split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'split3_train_score', 'mean_train_score', 'std_train_score', 'mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_model__max_depth', 'param_model__max_leaf_nodes', 'param_model__min_samples_leaf', 'param_model__n_estimators', 'params'])

#### Для оценки лучших параметров, посчитаем дисперсию, что бы понять какие параметры дают более стабильную модель, т.е. оценивать будем не только по результатм метрики 

In [536]:
rf_gs_res = pd.DataFrame()
rf_gs_res['0'] = gridsearch.cv_results_['split0_test_score']
rf_gs_res['1'] = gridsearch.cv_results_['split1_test_score']
rf_gs_res['2'] = gridsearch.cv_results_['split2_test_score']
rf_gs_res['3'] = gridsearch.cv_results_['split3_test_score']
rf_gs_res['P'] = gridsearch.cv_results_['params']
rf_gs_res['M'] = gridsearch.cv_results_['mean_test_score']

In [537]:
rf_gs_res['var'] = rf_gs_res[['0','1','2','3']].var(1)
rf_gs_res['var'].head()

0    0.000219
1    0.000187
2    0.000238
3    0.000172
4    0.000253
Name: var, dtype: float64

In [539]:
rf_gs_res[['M','P','var','0','1','2','3']].sort_values('var',  ascending = True) # try to find as best score as lowest var

,M,P,var,0,1,2,3
55,0.810325,"{'model__max_depth': 6, 'model__max_leaf_nodes...",0.000005,0.807175,0.811659,0.811659,0.810811
66,0.809203,"{'model__max_depth': 6, 'model__max_leaf_nodes...",0.000008,0.811659,0.807175,0.811659,0.806306
68,0.806958,"{'model__max_depth': 6, 'model__max_leaf_nodes...",0.000014,0.811659,0.807175,0.802691,0.806306
61,0.812570,"{'model__max_depth': 6, 'model__max_leaf_nodes...",0.000017,0.807175,0.816143,0.811659,0.815315
54,0.812570,"{'model__max_depth': 6, 'model__max_leaf_nodes...",0.000017,0.807175,0.811659,0.816143,0.815315
60,0.813692,"{'model__max_depth': 6, 'model__max_leaf_nodes...",0.000019,0.807175,0.816143,0.816143,0.815315
67,0.812570,"{'model__max_depth': 6, 'model__max_leaf_nodes...",0.000019,0.816143,0.807175,0.816143,0.810811
63,0.812570,"{'model__max_depth': 6, 'model__max_leaf_nodes...",0.000019,0.816143,0.816143,0.807175,0.810811
62,0.812570,"{'model__max_depth': 6, 'model__max_leaf_nodes...",0.000019,0.816143,0.816143,0.807175,0.810811
56,0.812570,"{'model__max_depth': 6, 'model__max_leaf_nodes...",0.000019,0.816143,0.816143,0.807175,0.810811


In [540]:
rf_gs_res.P.iloc[55] # the best of best

{'model__max_depth': 6,
 'model__max_leaf_nodes': 50,
 'model__min_samples_leaf': 1,
 'model__n_estimators': 170}

In [51]:
#sorted(gridsearch.cv_results_['mean_test_score'][:10], reverse=True)

In [52]:
#for i in range(1,len(gridsearch.cv_results_['mean_test_score'][:60])):
#    print(gridsearch.cv_results_['mean_test_score'][i],gridsearch.cv_results_['params'][i] )

In [541]:
pipline_gb=Pipeline([('scale', StandardScaler()), 
                  ('model', GradientBoostingClassifier(n_estimators=200, 
                                                   #loss='exponential', 
                                                   #learning_rate =0.1,
                                                   random_state = 5,    
                                                   max_depth = 3
                                                    ))])

grid_params2 = {'model__max_depth': [2,3,4], 
         #'model__loss' : ['deviance', 'exponential'],
         #'model__learning_rate': [0.03,0.05],
         #'model__min_samples_split': [2,3,4],
         #'model__max_leaf_nodes': max_leaf_nodes,
         #'model__min_samples_leaf': [1,2,5] , 
         #'model__min_weight_fraction_leaf': min_weight_fraction_leaf,
         #'model__max_features' : [0.3,0.8, 'auto', 1],
         'model__n_estimators' : [100,150,200,210]
         }  

gridsearch2 = GridSearchCV(pipline_gb, grid_params2, scoring='accuracy', cv=kfold_out)

In [542]:
gridsearch2.fit(X_train, y)

GridSearchCV(cv=KFold(n_splits=4, random_state=None, shuffle=True),
       error_score='raise',
       estimator=Pipeline(steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('model', GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=...ors=200, presort='auto', random_state=5,
              subsample=1.0, verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'model__max_depth': [2, 3, 4], 'model__n_estimators': [100, 150, 200, 210]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [544]:
gb_gs_res = pd.DataFrame()
gb_gs_res['0'] = gridsearch2.cv_results_['split0_test_score']
gb_gs_res['1'] = gridsearch2.cv_results_['split1_test_score']
gb_gs_res['2'] = gridsearch2.cv_results_['split2_test_score']
gb_gs_res['3'] = gridsearch2.cv_results_['split3_test_score']
#gb_gs_res['4'] = gridsearch2.cv_results_['split4_test_score']
#gb_gs_res['5'] = gridsearch2.cv_results_['split5_test_score']
gb_gs_res['P'] = gridsearch2.cv_results_['params']
gb_gs_res['M'] = gridsearch2.cv_results_['mean_test_score']

In [545]:
gb_gs_res['var'] = gb_gs_res[['0','1','2','3']].var(1)
gb_gs_res['var'].head()

0    0.000382
1    0.000172
2    0.000230
3    0.000086
4    0.000206
Name: var, dtype: float64

In [546]:
gb_gs_res[['M','P','var']].sort_values('var',ascending = True)

,M,P,var
9,0.835017,"{'model__max_depth': 4, 'model__n_estimators':...",0.000080
3,0.826038,"{'model__max_depth': 2, 'model__n_estimators':...",0.000086
11,0.838384,"{'model__max_depth': 4, 'model__n_estimators':...",0.000128
10,0.836139,"{'model__max_depth': 4, 'model__n_estimators':...",0.000147
8,0.833895,"{'model__max_depth': 4, 'model__n_estimators':...",0.000166
1,0.824916,"{'model__max_depth': 2, 'model__n_estimators':...",0.000172
6,0.822671,"{'model__max_depth': 3, 'model__n_estimators':...",0.000193
4,0.824916,"{'model__max_depth': 3, 'model__n_estimators':...",0.000206
5,0.828283,"{'model__max_depth': 3, 'model__n_estimators':...",0.000215
2,0.824916,"{'model__max_depth': 2, 'model__n_estimators':...",0.000230


In [547]:
gb_gs_res.P.iloc[9] # the best 

{'model__max_depth': 4, 'model__n_estimators': 150}

In [79]:
#sorted(gridsearch2.cv_results_['mean_test_score'][:10], reverse=True)

In [127]:
#for i in range(0,len(gridsearch2.cv_results_['mean_test_score'][:90])):
#    print(gridsearch2.cv_results_['mean_test_score'][i],gridsearch2.cv_results_['params'][i]['model__learning_rate'],gridsearch2.cv_results_['params'][i]['model__max_depth'],gridsearch2.cv_results_['params'][i]['model__n_estimators']  )

In [548]:
gridsearch2.best_params_

{'model__max_depth': 4, 'model__n_estimators': 210}

In [231]:
# Плохой результат, но я не делал StandartScaler потому как оставил категорийные фичи

grid_params_cat = {'depth': [3,4,5,6,7,8], 
         #'model__loss' : ['deviance', 'exponential'],
         'learning_rate': [ 0.05,0.08,0.1],
         #'max_leaf_nodes': max_leaf_nodes,
         #'min_samples_leaf': min_samples_leaf , 
         # 'min_weight_fraction_leaf': min_weight_fraction_leaf,
         #'n_estimators' : [80,90, 100,110]
                  }  

gridsearch_cat = GridSearchCV(CatBoostClassifier(cat_features=[0, 1, 6]), grid_params_cat, scoring='accuracy', cv=kfold_out)

In [75]:
gridsearch_cat.fit(Xcat_train.as_matrix(),y)

GridSearchCV(cv=KFold(n_splits=4, random_state=22, shuffle=True),
       error_score='raise',
       estimator=<catboost.core.CatBoostClassifier object at 0x000000000C1F7518>,
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'depth': [3, 4, 5, 6, 7, 8], 'learning_rate': [0.05, 0.08, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [76]:
for i in range(1,len(gridsearch_cat.cv_results_['mean_test_score'][:40])):
    print(gridsearch_cat.cv_results_['mean_test_score'][i],gridsearch_cat.cv_results_['params'][i] )

0.820426487093 {'depth': 3, 'learning_rate': 0.08}
0.817059483726 {'depth': 3, 'learning_rate': 0.1}
0.82379349046 {'depth': 4, 'learning_rate': 0.05}
0.810325476992 {'depth': 4, 'learning_rate': 0.08}
0.812570145903 {'depth': 4, 'learning_rate': 0.1}
0.812570145903 {'depth': 5, 'learning_rate': 0.05}
0.811447811448 {'depth': 5, 'learning_rate': 0.08}
0.813692480359 {'depth': 5, 'learning_rate': 0.1}
0.814814814815 {'depth': 6, 'learning_rate': 0.05}
0.808080808081 {'depth': 6, 'learning_rate': 0.08}
0.796857463524 {'depth': 6, 'learning_rate': 0.1}
0.806958473625 {'depth': 7, 'learning_rate': 0.05}
0.806958473625 {'depth': 7, 'learning_rate': 0.08}
0.799102132435 {'depth': 7, 'learning_rate': 0.1}
0.809203142536 {'depth': 8, 'learning_rate': 0.05}
0.796857463524 {'depth': 8, 'learning_rate': 0.08}
0.799102132435 {'depth': 8, 'learning_rate': 0.1}


In [549]:
#Плохие результаты не улучшает модель
pipline_gp=Pipeline([('scale', StandardScaler()), 
                  ('model', GaussianProcessClassifier( ))])

grid_params_gp = {'model__n_restarts_optimizer': [0,3,4,5], 
         #'model__loss' : ['deviance', 'exponential'],
         'model__max_iter_predict': [90,100,110,120,150]}  

gridsearch_gp = GridSearchCV(pipline_gp, grid_params_gp, scoring='accuracy', cv=kfold_out)

In [550]:
gridsearch_gp.fit(X_train, y)

GridSearchCV(cv=KFold(n_splits=4, random_state=None, shuffle=True),
       error_score='raise',
       estimator=Pipeline(steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('model', GaussianProcessClassifier(copy_X_train=True, kernel=None,
             max_iter_predict=100, multi_class='one_vs_rest', n_jobs=1,
             n_restarts_optimizer=0, optimizer='fmin_l_bfgs_b',
             random_state=None, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'model__n_restarts_optimizer': [0, 3, 4, 5], 'model__max_iter_predict': [90, 100, 110, 120, 150]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [551]:
gridsearch_gp.cv_results_['mean_test_score']

array([ 0.81257015,  0.81257015,  0.81257015,  0.81257015,  0.81257015,
        0.81257015,  0.81257015,  0.81257015,  0.81257015,  0.81257015,
        0.81257015,  0.81257015,  0.81257015,  0.81257015,  0.81257015,
        0.81257015,  0.81257015,  0.81257015,  0.81257015,  0.81257015])

In [232]:
from sklearn.linear_model import LogisticRegression

In [233]:
pipline_logreg=Pipeline([('scale', StandardScaler()), 
                  ('model', LogisticRegression())])

loggrid = {'penalty' : ['l1','l2'], 'C': [0.01,0.05, 0.1, 1, 5,10 ], 'class_weight' : [None,'balanced'] }
loggridsearch=GridSearchCV(LogisticRegression(), loggrid, scoring='accuracy', cv=5)

In [234]:
loggridsearch.fit(X_train, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.01, 0.05, 0.1, 1, 5, 10], 'class_weight': [None, 'balanced']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [235]:
loggridsearch.best_params_

{'C': 1, 'class_weight': None, 'penalty': 'l2'}

In [236]:
loggridsearch.best_score_

0.79461279461279466

In [239]:
mycat=CatBoostClassifier(depth=4, learning_rate=0.05)

In [240]:
Xcat_train = X_cat.loc[X.is_test==False]
Xcat_test = X_cat.loc[X.is_test==True]

In [241]:
del Xcat_test['is_test']
del Xcat_train['is_test']

In [242]:
Xcat_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 7 columns):
Pclass      891 non-null int32
Sex         891 non-null int32
Age         891 non-null float64
SibSp       891 non-null float64
Parch       891 non-null float64
Fare        891 non-null float64
Embarked    891 non-null int32
dtypes: float64(4), int32(3)
memory usage: 45.2 KB


In [243]:
Xcat_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 891 to 1308
Data columns (total 7 columns):
Pclass      418 non-null int32
Sex         418 non-null int32
Age         418 non-null float64
SibSp       418 non-null float64
Parch       418 non-null float64
Fare        418 non-null float64
Embarked    418 non-null int32
dtypes: float64(4), int32(3)
memory usage: 21.2 KB


In [244]:
mycat.fit(Xcat_train.as_matrix(), y, cat_features=[0, 1, 6])

In [245]:
cat_test_pred = mycat.predict_proba(Xcat_test.as_matrix())

In [246]:
cat_pred_test = mycat.predict(Xcat_test.as_matrix()).astype(int)

In [247]:
cat_pred_train=mycat.predict(Xcat_train.as_matrix()).astype(int)

In [248]:
accuracy_score(cat_pred_train, y)

0.8630751964085297

In [249]:
#Результирующий лес для стэкинга, использую не совсем те параметры что в тетрадке, потому как от запуска к запуску они отличаются.
#это параметры от того результата который я отправлял на kaggle
pipline_rf_res=Pipeline([('scale', StandardScaler()), 
                  ('model', RandomForestClassifier(n_estimators=170, 
                                                   #loss='deviance', 
                                                   #learning_rate =0.1,
                                                   max_leaf_nodes=100,
                                                   #max_features = 0.8,
                                                   max_depth = 6
                                                  ))])

In [250]:
#результирующий gradient boosting для стэкинга
pipline_gb_res=Pipeline([('scale', StandardScaler()), 
                  ('model', GradientBoostingClassifier(n_estimators=200, 
                                                   #loss='exponential', 
                                                   #learning_rate = 0.05,
                                                   #max_leaf_nodes = None,
                                                   #min_samples_leaf = 2,    
                                                   #min_samples_split = 4,
                                                   #max_features = 1, 
                                                   random_state =5,    
                                                   max_depth = 3   
                                                      ))])

In [251]:
pipline_cat_res=Pipeline([('scale', StandardScaler()), 
                  ('model', CatBoostClassifier(n_estimators=110, 
                                                   loss='exponential', 
                                                   learning_rate =0.01,
                                                   max_depth = 4  ))])

In [252]:
pipline_nb_res=Pipeline([('scale', StandardScaler()), 
                  ('model', GaussianNB( ))])

In [253]:
pipline_gp_res=Pipeline([('scale', StandardScaler()), 
                  ('model', GaussianProcessClassifier( ))])

In [254]:
pipline_logreg_res=Pipeline([('scale', StandardScaler()), 
                             ('model', LogisticRegression(C=1, penalty='l2' ))])

In [255]:
y.shape

(891,)

In [256]:
pipline_svd_res=Pipeline([('scale', StandardScaler()), 
                  ('model', SGDClassifier(loss="log", alpha=1/891, random_state=42) )])

Пробовал разные комбинации, разных моделей, в большинстве случаев или становилось сильно хуже или какой то алгоритм использовался а все остальные обнулялись, пробовал так же увелчить n_splits в функции разбиения - не дало результата

In [307]:
# получаем предсказания вероятностей ансамблей на кросс-валидации для обучающей выборки
rf_train_pred = cross_val_predict_proba(pipline_rf_res, X_train, y)
gb_train_pred = cross_val_predict_proba(pipline_gb_res, X_train, y)
nb_train_pred = cross_val_predict_proba(pipline_nb_res, X_train, y)
gp_train_pred = cross_val_predict_proba(pipline_gp_res, X_train, y)
svd_train_pred = cross_val_predict_proba(pipline_svd_res, X_train, y)
log_train_pred = cross_val_predict_proba(pipline_logreg_res, X_train, y)
#cat_train_pred = cross_val_predict_proba(mycat, X_train, y)
cat_train_pred = cross_val_predict_proba(mycat, Xcat_train.as_matrix(), y)

gp_clf = pipline_gb_res.fit(X_train,y)
gp_pred_test = gp_clf.predict(X_test)
gp_pred_train = gp_clf.predict(X_train)
rf_clf = pipline_rf_res.fit(X_train,y)
rf_pred_test = rf_clf.predict(X_test)
nb_pred_train = pipline_nb_res.fit(X_train,y).predict(X_train)
nb_pred_test = pipline_nb_res.predict(X_test)
#X_train_stack = np.stack([rf_train_pred[:,1], gb_train_pred[:,1], cat_train_pred[:,1]], axis=1)
#X_train_stack = np.stack([rf_train_pred[:,1], gb_train_pred[:,1], gp_train_pred[:,1], nb_train_pred[:,1]], axis=1)
#X_train_stack = np.stack([rf_train_pred[:,1], gb_train_pred[:,1]], axis=1)

X_train_stack = np.stack([nb_train_pred[:,1], svd_train_pred[:,1], log_train_pred[:,1]], axis=1)
#all_train_pred = np.stack([gb_train_pred[:,1], gp_train_pred[:,1],  nb_train_pred[:,1], svd_train_pred[:,1], cat_train_pred[:,1], log_train_pred[:,1]], axis=1)

# получаем предсказания ансамблей для тестовой выборки
rf_test_pred = pipline_rf_res.fit(X_train, y).predict_proba(X_test)
gb_test_pred = pipline_gb_res.fit(X_train, y).predict_proba(X_test)
nb_test_pred = pipline_nb_res.fit(X_train, y).predict_proba(X_test)
gp_test_pred = pipline_gp_res.fit(X_train, y).predict_proba(X_test)
svd_test_pred = pipline_svd_res.fit(X_train, y).predict_proba(X_test)
cat_test_pred = mycat.predict_proba(Xcat_test.as_matrix())
log_test_pred = pipline_logreg_res.fit(X_train, y).predict_proba(X_test)


#cat_test_pred = mycat.fit(Xcat_train.as_matrix(), y).predict_proba(Xcat_test.as_matrix())

#X_test_stack = np.stack([rf_test_pred[:,1], gb_test_pred[:,1],cat_test_pred[:,1]], axis=1)
#X_test_stack = np.stack([rf_test_pred[:,1], gb_test_pred[:,1]], axis=1)
X_test_stack = np.stack([ nb_test_pred[:,1], svd_test_pred[:,1], log_test_pred[:,1] ], axis=1)
#all_test_pred = np.stack([gb_test_pred[:,1], gp_test_pred[:,1],  nb_test_pred[:,1], svd_test_pred[:,1],cat_test_pred[:,1], log_test_pred[:,1]], axis=1)

In [341]:
X_test_stack = np.stack([rf_test_pred[:,1], nb_test_pred[:,1], svd_test_pred[:,1], log_test_pred[:,1] ], axis=1)
X_train_stack = np.stack([rf_train_pred[:,1] , nb_train_pred[:,1], svd_train_pred[:,1], log_train_pred[:,1]], axis=1)

In [448]:
X_test_stack = np.stack([rf_test_pred[:,1], gb_test_pred[:,1] ], axis=1)
X_train_stack = np.stack([rf_train_pred[:,1], gb_train_pred[:,1]], axis=1)

##### Лучший набор для стекинга это RF+GB но такой вариант просто ухудшает результат RF
#### Просьба дать комментарии. Т.к так и не получилось улучшить результат стэкингом. Может дело в переобучении GB? или в корреляции предсказаний? может стоило не подбирать параметры бустинга а взять например n_estimators = 20 ? или не брать градиентный бустинг раз он пееобучается?

In [449]:
# для первичной оценки
gb_pred_train = gp_clf.predict(X_train)
rf_pred_train = rf_clf.predict(X_train)
svd_pred_train = pipline_svd_res.predict(X_train)

In [450]:
#Для оцнки корреляций между предсказаниями
all_train_pred = np.stack([ gp_train_pred[:,1],  nb_train_pred[:,1], svd_train_pred[:,1], cat_train_pred[:,1], log_train_pred[:,1]], axis=1)
all_test_pred = np.stack([ gp_test_pred[:,1],  nb_test_pred[:,1], svd_test_pred[:,1], cat_test_pred[:,1], log_test_pred[:,1]], axis=1)

In [451]:
all_test_pred.shape

(418, 5)

In [452]:
all_train_pred.shape

(891, 5)

## Объединяем предсказания 

In [453]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [454]:
loggrid = {'penalty' : ['l1','l2'], 'C': [0.01,0.05, 0.1, 1, 5,10 ], 'class_weight' : [None,'balanced'] }

In [455]:
loggridsearch=GridSearchCV(LogisticRegression(), loggrid, scoring='accuracy', cv=5)

In [456]:
all_train_pred[:5,:]

array([[  1.88968796e-01,   1.27141138e-07,   3.00793074e-02,
          1.51801078e-01,   8.93631949e-02],
       [  8.31099513e-01,   2.20989377e-01,   7.54635999e-01,
          9.94477505e-01,   9.04374774e-01],
       [  4.62718902e-01,   1.09266105e-04,   5.30354279e-01,
          2.45613725e-01,   6.22092394e-01],
       [  8.42912591e-01,   8.08641704e-03,   5.44078134e-01,
          9.97252111e-01,   8.97995158e-01],
       [  1.47516562e-01,   4.23145414e-08,   7.09419680e-02,
          1.85336159e-01,   7.34690329e-02]])

In [553]:
#стэкинг через дерево работает лучше
treegrid = {'max_depth': [3,4,6,7,8], 
        #'max_leaf_nodes': max_leaf_nodes,
       #'min_samples_leaf': [1,2,5], 
       #'min_samples_split': [2,3,4] , 
        #'min_weight_fraction_leaf': min_weight_fraction_leaf,
       'class_weight': class_weight,
         #'max_features' : [ 0.5,0.8, 'auto']
           } 
treegridsearch = GridSearchCV(DecisionTreeClassifier(), treegrid, scoring='accuracy', cv= kfold_out)
#убрал параметры т.к. дерево переобучается, и берет в результат по сути один алгоритм

In [554]:
treegridsearch.fit(X_train_stack,y)

GridSearchCV(cv=KFold(n_splits=4, random_state=None, shuffle=True),
       error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': [3, 4, 6, 7, 8], 'class_weight': ['balanced', None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [555]:
treegridsearch.best_score_

0.80920314253647585

In [556]:
treegridsearch.best_params_

{'class_weight': None, 'max_depth': 3}

In [462]:
X_train_stack[:10,:10]

array([[ 0.1338317 ,  0.06227307],
       [ 0.98880525,  0.96862204],
       [ 0.44571255,  0.62281807],
       [ 0.98441466,  0.98425509],
       [ 0.11785761,  0.17627565],
       [ 0.11054155,  0.11005984],
       [ 0.2501582 ,  0.19091181],
       [ 0.4421471 ,  0.06451252],
       [ 0.48960251,  0.43400876],
       [ 0.93628007,  0.95293097]])

In [ ]:
pd.DataFrame(all_train_pred).corr()  # не смотря на то что logReg хорошо корелирует с другими предсказаниями результат улучшился, может дело и не в кореляции

In [463]:
loggridsearch.fit(X_train_stack, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.01, 0.05, 0.1, 1, 5, 10], 'class_weight': [None, 'balanced']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [464]:
for i in range(0,len(loggridsearch.cv_results_['mean_test_score'])):
    print(loggridsearch.cv_results_['mean_test_score'][i],loggridsearch.cv_results_['params'][i])

0.616161616162 {'C': 0.01, 'class_weight': None, 'penalty': 'l1'}
0.82379349046 {'C': 0.01, 'class_weight': None, 'penalty': 'l2'}
0.616161616162 {'C': 0.01, 'class_weight': 'balanced', 'penalty': 'l1'}
0.808080808081 {'C': 0.01, 'class_weight': 'balanced', 'penalty': 'l2'}
0.828282828283 {'C': 0.05, 'class_weight': None, 'penalty': 'l1'}
0.829405162738 {'C': 0.05, 'class_weight': None, 'penalty': 'l2'}
0.822671156004 {'C': 0.05, 'class_weight': 'balanced', 'penalty': 'l1'}
0.819304152637 {'C': 0.05, 'class_weight': 'balanced', 'penalty': 'l2'}
0.829405162738 {'C': 0.1, 'class_weight': None, 'penalty': 'l1'}
0.830527497194 {'C': 0.1, 'class_weight': None, 'penalty': 'l2'}
0.82379349046 {'C': 0.1, 'class_weight': 'balanced', 'penalty': 'l1'}
0.822671156004 {'C': 0.1, 'class_weight': 'balanced', 'penalty': 'l2'}
0.830527497194 {'C': 1, 'class_weight': None, 'penalty': 'l1'}
0.832772166105 {'C': 1, 'class_weight': None, 'penalty': 'l2'}
0.819304152637 {'C': 1, 'class_weight': 'balanced', 

In [465]:
loggridsearch.best_params_

{'C': 1, 'class_weight': None, 'penalty': 'l2'}

In [466]:
logreg = LogisticRegression(penalty='l1', C=0.05).fit(X_train_stack, y)

In [467]:
logreg.coef_  #Посмотрим кто вносит больший вклад в результат, видно что бустинг

array([[ 1.09812115,  2.14507825]])

In [468]:
restree = DecisionTreeClassifier(max_depth=3)

In [469]:
restree.fit(X_train_stack,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [470]:
restree.feature_importances_ # результат - Random Forest + GradientBoosting

array([ 0.91429256,  0.08570744])

#### Странно но DecisionTree отдает предпочтение RF,  лучшему в данной ситуации

In [471]:
from sklearn.tree import export_graphviz

def get_tree_dot_view(clf, feature_names=None, class_names=None):
    print(export_graphviz(clf, out_file=None, filled=True, feature_names=feature_names, class_names=class_names))

In [472]:
get_tree_dot_view(restree, [ 'gp',  'nb_train_pred[:,1]', 'svd_train_pred[:,1]', 'cat_train_pred[:,1]', 'log'], ['died','survived'])

digraph Tree {
node [shape=box, style="filled", color="black"] ;
0 [label="gp <= 0.4953\ngini = 0.473\nsamples = 891\nvalue = [549, 342]\nclass = died", fillcolor="#e5813960"] ;
1 [label="gp <= 0.2359\ngini = 0.286\nsamples = 613\nvalue = [507, 106]\nclass = died", fillcolor="#e58139ca"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="gp <= 0.0908\ngini = 0.1734\nsamples = 438\nvalue = [396, 42]\nclass = died", fillcolor="#e58139e4"] ;
1 -> 2 ;
3 [label="gini = 0.3029\nsamples = 129\nvalue = [105, 24]\nclass = died", fillcolor="#e58139c5"] ;
2 -> 3 ;
4 [label="gini = 0.1097\nsamples = 309\nvalue = [291, 18]\nclass = died", fillcolor="#e58139ef"] ;
2 -> 4 ;
5 [label="nb_train_pred[:,1] <= 0.1118\ngini = 0.4639\nsamples = 175\nvalue = [111, 64]\nclass = died", fillcolor="#e581396c"] ;
1 -> 5 ;
6 [label="gini = 0.0713\nsamples = 27\nvalue = [26, 1]\nclass = died", fillcolor="#e58139f5"] ;
5 -> 6 ;
7 [label="gini = 0.489\nsamples = 148\nvalue = [85, 63]\nclass = d

In [1]:
from IPython.display import display, HTML
display(HTML('''<img src="data:image/jpeg;base64,/511546863f2c97480be7c5f7161439e914feb5f0//tree.png">'''))

![text](tree.png)

#### как видно из картинки, немного улучшили предсказание выживших.

In [473]:
predicted = logreg.predict(X_test_stack)
predicted_tree = restree.predict(X_test_stack)
#predicted  = gridsearch2.best_estimator_.predict(X_test)
predicted_train=logreg.predict(X_train_stack)
predicted_tree_train = restree.predict(X_train_stack)

In [438]:
accuracy_score(predicted_train,y)

0.81818181818181823

In [474]:
accuracy_score(predicted_tree_train, y)

0.84062850729517391

In [475]:
accuracy_score(y,gb_pred_train)

0.92255892255892258

In [476]:
accuracy_score(y,rf_pred_train)

0.89113355780022452

In [477]:
accuracy_score(y,nb_pred_train)

0.64646464646464652

In [478]:
accuracy_score(y,gp_pred_train)

0.92255892255892258

In [479]:
accuracy_score(y,svd_pred_train)

0.78900112233445563

## Формируем файл

In [480]:
with open('submission_ensamble_titanic.txt', 'w') as out:
    out.write('PassengerId,Survived\n')
    for passenger, y_iter in zip(test['PassengerId'], predicted_tree):
        out.write('%s,%s\n' % (passenger, y_iter))

In [288]:
## Result - 78947 -  rf + gb

In [289]:
## Best estimator - RandomForest -0.79904